# Temperature dependent elastic constants

## Background

$$C_{ijkl} = \frac{1}{V} \frac{\partial^2 U}{\partial \varepsilon_{ij}\partial \varepsilon_{kl}}$$

$$U(T) = \frac{V}{2}C_{ijkl}(T)\varepsilon_{ij}\varepsilon_{kl}$$

$$\sigma_{ij} = C_{ijkl}{\varepsilon_{kl}}$$

### How to get $U$ or $\sigma$

- MD
- Quasi-Harmonic

## Tasks

- Get $a_0$ from potential
- Lattice parameter (as a function of T)
  - MD
    - NVT
    - NPT
  - QH
- Calculate $U$ or $\sigma$ for various $\varepsilon$
  - MD: Equilibriate and average with LAMMPS
  - QH: Get strains from Yuriy's tool and run phonopy
- Fit

## Teams

- MD: Erik, Han, (Raynol), Prabhath, Jan
- QH: Raynol, (Sam), Bharathi, Ahmed, Haitham
- Fit & Yuriy: Sam
- Literature

In [1]:
from ase.build import bulk
from ase.atoms import Atoms

In [2]:
import numpy as np
import os

from atomistics.workflows.elastic.workflow import (
    analyse_structures_helper,
    generate_structures_helper,
)

from atomistics.calculators import evaluate_with_lammpslib, get_potential_by_name

In [23]:
structure = bulk('Al', 'fcc', a=4.05, cubic=True)

In [4]:
potential_name = "1999--Mishin-Y--Al--LAMMPS--ipr1"

In [ ]:
def get_minimum_lattice_constant(structure: Atoms, potential: str) -> float:

    structure_relaxed = get_relaxed_structure(structure, potential)
    a_0 = structure_relaxed.get_volume()**(1/3) #Angstrom

    return a_0

In [ ]:
def get_relaxed_structure(structure: Atoms, potential: str) -> Atoms:
    
    df_pot_selected = get_potential_by_name(
            potential_name=potential
        )
    
    result_dict = evaluate_with_lammpslib(
            task_dict={"optimize_positions_and_volume": structure},
            potential_dataframe=df_pot_selected,
        )
    
    structure_relaxed = result_dict['structure_with_optimized_positions_and_volume']

    return structure_relaxed

In [10]:
a_0 = get_minimum_lattice_constant(structure, potential_name)
a_0

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


4.050004662201837

In [9]:
relaxed_structure = get_relaxed_structure(structure, potential_name)
relaxed_structure

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


Atoms(symbols='Al4', pbc=True, cell=[4.050004662201837, 4.050004662201837, 4.050004662201837])

```
def get_lattice_constant_with_QH(
    structure: "ase.atoms.Atoms",
    temperature: list[float] | float,
    engine,
    **kwargs,
) -> list[float] | float:
    ...
    return a_0
```

* https://atomistics.readthedocs.io/en/latest/bulk_modulus_with_gpaw.html#elastic-matrix
* https://github.com/pyiron/atomistics/blob/main/tests/test_elastic_lammpslib_functional.py
* https://github.com/pyiron/pyiron_workflow_atomistics/blob/interstitials/pyiron_workflow_atomistics/dataclass_storage.py
* https://github.com/ligerzero-ai/pyiron_workflow_lammps/blob/main/pyiron_workflow_lammps/engine.py#L21

In [ ]:
def get_lattice_constant_with_MD_NPT(
    structure: Atoms,
    temperature: list[float] | float,
    engine,
    **kwargs,
) -> list[float] | float:
    ...
    return a_0

In [ ]:
def get_lattice_constant_with_MD_NVT(
    structure: "ase.atoms.Atoms",
    temperature: list[float] | float,
    engine,
    **kwargs,
) -> list[float] | float:
    ...
    return a_0

In [ ]:
def get_deformations(structure) -> list[list[float, float, float, float, float, float]]:
    ...
    return epsilon

In [ ]:
# distribute get_stress_with_MD

def get_stress_with_MD(
    structure, temperature, strains: list[float, float, float, float, float, float], engine
):
    ...
    return sigma

def get_energy_with_MD(structure, temperature, strains, engine):
    ...
    return energy

def get_stress_with_QH(structure, temperature, strains, engine):
    ...
    return sigma

def get_energy_with_QH(structure, temperature, strains, engine):
    ...
    return energy

In [24]:
potential = potential_name

sym_dict, structure_dict = generate_structures_helper(
    structure=relaxed_structure,
    eps_range=0.005,
    num_of_point=5,
    zero_strain_job_name="s_e_0",
    sqrt_eta=True,
)
structure_dict

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/workflows/elastic/symmetry.py:32: DeprecationWarning: dict interface is deprecated. Use attribute interface instead
  SGN = dataset["number"]


OrderedDict([('s_e_0',
              Atoms(symbols='Al4', pbc=True, cell=[4.050004662201837, 4.050004662201837, 4.050004662201837])),
             ('s_01_e_m0_00500',
              Atoms(symbols='Al4', pbc=True, cell=[4.0297037591141, 4.0297037591141, 4.0297037591141])),
             ('s_01_e_m0_00250',
              Atoms(symbols='Al4', pbc=True, cell=[4.039866962542076, 4.039866962542076, 4.039866962542076])),
             ('s_01_e_0_00250',
              Atoms(symbols='Al4', pbc=True, cell=[4.060117049134704, 4.060117049134704, 4.060117049134704])),
             ('s_01_e_0_00500',
              Atoms(symbols='Al4', pbc=True, cell=[4.07020431200885, 4.07020431200885, 4.07020431200885])),
             ('s_08_e_m0_00500',
              Atoms(symbols='Al4', pbc=True, cell=[4.0297037591141, 4.0297037591141, 4.050004662201837])),
             ('s_08_e_m0_00250',
              Atoms(symbols='Al4', pbc=True, cell=[4.039866962542076, 4.039866962542076, 4.050004662201837])),
             ('s_

## Calculate C11

In [45]:
structure_strained = relaxed_structure.copy()

structure_strained.set_cell(
    [structure_strained.get_cell()[0]*(0.995), structure_strained.get_cell()[1], structure_strained.get_cell()[2]],
    scale_atoms=True
)

structure_strained

Atoms(symbols='Al4', pbc=True, cell=[4.029754638890828, 4.050004662201837, 4.050004662201837])

In [46]:
relaxed_dict = calc_static_with_lammpslib(structure=relaxed_structure, potential_dataframe= df_pot_selected)
strained_dict = calc_static_with_lammpslib(structure=structure_strained, potential_dataframe= df_pot_selected)

print("Relaxed stress:\n", relaxed_dict['stress'])
print("Strained stress:\n", strained_dict['stress'])

diff = strained_dict['stress'] - relaxed_dict['stress']
C11 = diff[0, 0] / 0.005
# C12 = diff[0, 1] / 0.005

C11

Relaxed stress:
 [[-2.43580132e-04  2.02497379e-11  2.52703340e-11]
 [ 2.02497379e-11 -2.43580106e-04  2.62291021e-11]
 [ 2.52703340e-11  2.62291021e-11 -2.43580114e-04]]
Strained stress:
 [[ 5.72921753e+03 -1.46328933e-11 -1.22781749e-11]
 [-1.46328933e-11  3.07529760e+03  2.34727955e-11]
 [-1.22781749e-11  2.34727955e-11  3.07529760e+03]]


1145843.555110171

## Calculate C12

In [47]:
structure_strained = relaxed_structure.copy()

structure_strained.set_cell(
    [structure_strained.get_cell()[0]*(0.995), structure_strained.get_cell()[1]*(0.995), structure_strained.get_cell()[2]],
    scale_atoms=True
)

structure_strained

Atoms(symbols='Al4', pbc=True, cell=[4.029754638890828, 4.029754638890828, 4.050004662201837])

In [48]:
relaxed_dict = calc_static_with_lammpslib(structure=relaxed_structure, potential_dataframe= df_pot_selected)
strained_dict = calc_static_with_lammpslib(structure=structure_strained, potential_dataframe= df_pot_selected)

print("Relaxed stress:\n", relaxed_dict['stress'])
print("Strained stress:\n", strained_dict['stress'])

diff = strained_dict['stress'] - relaxed_dict['stress']
# C11 = diff[0, 0] / 0.005
# C12 = diff[0, 1] / 0.005

sigma11 = diff[0, 0]
sigma33 = diff[2, 2]

C12 = (sigma33/ 0.005) /2
C12

Relaxed stress:
 [[-2.43580132e-04  2.02497379e-11  2.52703340e-11]
 [ 2.02497379e-11 -2.43580106e-04  2.62291021e-11]
 [ 2.52703340e-11  2.62291021e-11 -2.43580114e-04]]
Strained stress:
 [[ 8.80299001e+03  1.47064255e-11 -2.02847248e-12]
 [ 1.47064255e-11  8.80299001e+03  7.45067521e-12]
 [-2.02847248e-12  7.45067521e-12  6.13579643e+03]]


613579.6671739953

In [49]:
sigma11/ 0.005 - C11

614754.4955247561

In [50]:
sigma22 = diff[1, 1]
sigma22/ 0.005 - C11

614754.4955247543

## Calculate C44

In [92]:
relaxed_structure.get_volume()

66.43035441556098

In [53]:
relaxed_structure.get_cell().tolist()

[[4.050004662201837, 0.0, 0.0],
 [0.0, 4.050004662201837, 0.0],
 [0.0, 0.0, 4.050004662201837]]

In [102]:
structure_strained = relaxed_structure.copy()
relaxed_cell = np.array(structure_strained.get_cell().tolist())

# F = np.eye(3,3) + np.array([
#     [ 0, 0, 0],
#     [ 0, 0, 0.0005 * relaxed_cell[2, 2]],
#     [ 0, 0.0005 * relaxed_cell[1, 1], 0]
# ])

F = np.eye(3,3) + np.array([
    [ 0, 0, 0],
    [ 0, 0, 0.0005],
    [ 0, 0.0005, 0]
])

strained_cell = F@relaxed_cell
print(strained_cell)

structure_strained.set_cell(
    strained_cell,
    scale_atoms=True
)

structure_strained

[[4.05000466e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 4.05000466e+00 2.02500233e-03]
 [0.00000000e+00 2.02500233e-03 4.05000466e+00]]


Atoms(symbols='Al4', pbc=True, cell=[[4.050004662201837, 0.0, 0.0], [0.0, 4.050004662201837, 0.0020250023311009185], [0.0, 0.0020250023311009185, 4.050004662201837]])

In [103]:
structure_strained.get_volume()

66.43033780797236

In [104]:
from structuretoolkit.visualize import plot3d

plot3d(structure_strained)

NGLWidget()

In [106]:
relaxed_dict = calc_static_with_lammpslib(structure=relaxed_structure, potential_dataframe= df_pot_selected)
strained_dict = calc_static_with_lammpslib(structure=structure_strained, potential_dataframe= df_pot_selected)

print("Relaxed stress:\n", relaxed_dict['stress'])
print("Strained stress:\n", strained_dict['stress'])

diff = strained_dict['stress'] - relaxed_dict['stress']
print("Stress difference:\n", diff)

sigma23 = diff[2, 1]

C44 = sigma23 / (2 *0.0005)
C44

Relaxed stress:
 [[-2.43580132e-04  2.02497379e-11  2.52703340e-11]
 [ 2.02497379e-11 -2.43580106e-04  2.62291021e-11]
 [ 2.52703340e-11  2.62291021e-11 -2.43580114e-04]]
Strained stress:
 [[ 4.10367247e-01 -3.37954779e-11 -1.97645834e-11]
 [-3.37954779e-11  3.47506713e-01 -3.15961812e+02]
 [-1.97645834e-11 -3.15961812e+02  3.47506713e-01]]
Stress difference:
 [[ 4.10610827e-01 -5.40452157e-11 -4.50349174e-11]
 [-5.40452157e-11  3.47750293e-01 -3.15961812e+02]
 [-4.50349174e-11 -3.15961812e+02  3.47750293e-01]]


-315961.8121188986

In [17]:
from atomistics.calculators.lammps.libcalculator import calc_static_with_lammpslib

In [18]:
calc_static_with_lammpslib?

Signature:
calc_static_with_lammpslib(
    structure: 'Atoms',
    potential_dataframe: 'pandas.DataFrame',
    lmp=None,
    output_keys=('forces', 'energy', 'stress', 'volume'),
    **kwargs,
) -> 'dict'
Docstring: <no docstring>
File:      /cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/libcalculator.py
Type:      function

In [19]:
df_pot_selected = get_potential_by_name(
    potential_name=potential
)


/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


In [ ]:
df_pot_selected = get_potential_by_name(
    potential_name=potential
)

result_dict = evaluate_with_lammpslib(
    task_dict={"calc_energy": structure_dict},
    potential_dataframe=df_pot_selected,
)

sym_dict, elastic_dict = analyse_structures_helper(
    output_dict=result_dict,
    sym_dict=sym_dict,
    fit_order=2,
    zero_strain_job_name="s_e_0",
)

In [12]:
def fit_elastic_constants(structure: Atoms, potential: str, strains, stresses=None, energies=None):

    sym_dict, structure_dict = generate_structures_helper(
        structure=structure,
        eps_range=0.005,
        num_of_point=5,
        zero_strain_job_name="s_e_0",
        sqrt_eta=True,
    )

    df_pot_selected = get_potential_by_name(
        potential_name=potential
    )

    result_dict = evaluate_with_lammpslib(
        task_dict={"calc_energy": structure_dict},
        potential_dataframe=df_pot_selected,
    )

    sym_dict, elastic_dict = analyse_structures_helper(
        output_dict=result_dict,
        sym_dict=sym_dict,
        fit_order=2,
        zero_strain_job_name="s_e_0",
    )

    return elastic_dict

In [14]:
relaxed_structure = get_relaxed_structure(structure, potential_name)
elast_dict = fit_elastic_constants(
    structure=relaxed_structure,
    potential=potential_name,
    strains=None)
elast_dict['elastic_matrix']

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/workflows/elastic/symmetry.py:32: DeprecationWarning: dict interface is deprecated. Use attribute interface instead
  SGN = dataset["number"]
/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

array([[114.103117  ,  60.51102935,  60.51102935,   0.        ,
          0.        ,   0.        ],
       [ 60.51102935, 114.103117  ,  60.51102935,   0.        ,
          0.        ,   0.        ],
       [ 60.51102935,  60.51102935, 114.103117  ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,  31.67489592,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         31.67489592,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  31.67489592]])

In [ ]:
def get_bulk_structure(
    name: str,
    crystalstructure=None,
    a=None,
    b=None,
    c=None,
    alpha=None,
    covera=None,
    u=None,
    orthorhombic=False,
    cubic=False,
    basis=None,
):
    from ase.build import bulk
    equil_struct = bulk(
        name=name,
        crystalstructure=crystalstructure,
        a=a,
        b=b,
        c=c,
        alpha=alpha,
        covera=covera,
        u=u,
        orthorhombic=orthorhombic,
        cubic=cubic,
        basis=basis,
    )
    return equil_struct